# Assignment filter


ojnsdvojsdnjdg

In [27]:
from pathlib import Path

import sympy as sp
import numpy as np
import xarray as xr
import xarray.ufuncs as xf
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

%matplotlib widget

from cw.filters import IteratedExtendedKalmanFilter
from system_identification.load_data import load_data

In [2]:
u, v, w, cau = sp.symbols("u, v, w, C_alpha_u")
alpha_m, beta_m, v_m = sp.symbols("alpha_m, beta_m, V_m")
u_dot, v_dot, w_dot = sp.symbols("udot, vdot, wdot")

asc;sdjvhdldbv
svjspndvj
sdvosdv $\alpha + \beta = \gamma$

In [3]:
iekf = IteratedExtendedKalmanFilter(
    x=[u, v, w, cau],
    z=[alpha_m, beta_m, v_m],
    u=[u_dot, v_dot, w_dot],
    f=sp.Matrix([
        u_dot,
        v_dot,
        w_dot,
        0.
    ]),
    h=sp.Matrix([
        sp.atan(w / u) * (1 + cau),
        sp.atan(v / sp.sqrt(u**2 + w**2)),
        sp.sqrt(u**2 + v**2 + w**2)
    ]),
    g=sp.Matrix(np.diag([1, 1, 1, 0])),
    max_iterations=100,
    eps=1e-10,
)
print(iekf)

IteratedExtendedKalmanFilter:
  x:
    (u, v, w, Cₐₗₚₕₐ ᵤ)
  z:
    (αₘ, βₘ, Vₘ)
  u:
    (u̇, v̇, ẇ)
  f:
    ⎡ u̇ ⎤
    ⎢   ⎥
    ⎢ v̇ ⎥
    ⎢   ⎥
    ⎢ ẇ ⎥
    ⎢   ⎥
    ⎣0.0⎦
  g:
    ⎡1  0  0  0⎤
    ⎢          ⎥
    ⎢0  1  0  0⎥
    ⎢          ⎥
    ⎢0  0  1  0⎥
    ⎢          ⎥
    ⎣0  0  0  0⎦
  h:
    ⎡                   ⎛w⎞⎤
    ⎢(Cₐₗₚₕₐ ᵤ + 1)⋅atan⎜─⎟⎥
    ⎢                   ⎝u⎠⎥
    ⎢                      ⎥
    ⎢      ⎛     v      ⎞  ⎥
    ⎢  atan⎜────────────⎟  ⎥
    ⎢      ⎜   _________⎟  ⎥
    ⎢      ⎜  ╱  2    2 ⎟  ⎥
    ⎢      ⎝╲╱  u  + w  ⎠  ⎥
    ⎢                      ⎥
    ⎢     ______________   ⎥
    ⎢    ╱  2    2    2    ⎥
    ⎣  ╲╱  u  + v  + w     ⎦
  fx:
    ⎡0  0  0  0⎤
    ⎢          ⎥
    ⎢0  0  0  0⎥
    ⎢          ⎥
    ⎢0  0  0  0⎥
    ⎢          ⎥
    ⎣0  0  0  0⎦
  hx:
    ⎡    -w⋅(Cₐₗₚₕₐ ᵤ + 1)                                          Cₐₗₚₕₐ ᵤ + 1             ⎛w⎞⎤
    ⎢    ──────────────────                  0                      ────────────    

In [4]:
iekf.hx

Matrix([
[           -w*(C_alpha_u + 1)/(u**2*(1 + w**2/u**2)),                                              0,                  (C_alpha_u + 1)/(u*(1 + w**2/u**2)), atan(w/u)],
[-u*v/((u**2 + w**2)**(3/2)*(v**2/(u**2 + w**2) + 1)), 1/(sqrt(u**2 + w**2)*(v**2/(u**2 + w**2) + 1)), -v*w/((u**2 + w**2)**(3/2)*(v**2/(u**2 + w**2) + 1)),         0],
[                          u/sqrt(u**2 + v**2 + w**2),                     v/sqrt(u**2 + v**2 + w**2),                           w/sqrt(u**2 + v**2 + w**2),         0]])

In [5]:
data_dir_path = Path().cwd().parent.parent / "assignment"
data = load_data(data_dir_path)

In [6]:
data

<xarray.Dataset>
Dimensions:  (t: 10001)
Coordinates:
  * t        (t) float64 0.0 0.01 0.02 0.03 0.04 ... 99.97 99.98 99.99 100.0
Data variables:
    alpha_m  (t) float64 -0.004572 -0.003229 -0.004428 ... -0.2303 -0.2288
    beta_m   (t) float64 0.002069 0.002634 0.0005577 ... 0.2579 0.2543 0.2574
    V_m      (t) float64 144.9 147.6 153.4 148.4 ... 152.3 152.1 152.1 153.9
    udot     (t) float64 1.043e-05 6.843e-06 4.02e-06 ... -2.244 -2.153 -2.05
    vdot     (t) float64 0.45 0.4499 0.4497 0.4494 ... 24.86 24.46 24.05 23.63
    wdot     (t) float64 2.855e-07 0.0225 0.04499 ... -49.15 -46.12 -43.07
    c_m      (t) float64 -0.05797 -0.05873 -0.05849 ... -0.03054 -0.02537

In [7]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(data.alpha_m, data.beta_m, data.c_m)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [46]:
f_data = iekf.filter(
    data=data,
    x_0=[150, 0, 0, 0.6021],
    p_0=np.diag([1.e-3, 1., 1., 1.]),
    q=np.diag([1e-3**2, 1e-3**2, 1e-3**2, 0.**2]),
    r=np.diag([0.01**2, 0.0058**2, 0.112**2])
)
f_data["vm_est"] = xf.sqrt(xf.square(f_data.u_est) + xf.square(f_data.v_est) + xf.square(f_data.w_est))

In [47]:
f_data

<xarray.Dataset>
Dimensions:        (dim_0: 4, dim_1: 4, t: 10001)
Coordinates:
  * t              (t) float64 0.0 0.01 0.02 0.03 ... 99.97 99.98 99.99 100.0
Dimensions without coordinates: dim_0, dim_1
Data variables:
    alpha_m        (t) float64 -0.004572 -0.003229 -0.004428 ... -0.2303 -0.2288
    beta_m         (t) float64 0.002069 0.002634 0.0005577 ... 0.2543 0.2574
    V_m            (t) float64 144.9 147.6 153.4 148.4 ... 152.1 152.1 153.9
    udot           (t) float64 1.043e-05 6.843e-06 4.02e-06 ... -2.153 -2.05
    vdot           (t) float64 0.45 0.4499 0.4497 0.4494 ... 24.46 24.05 23.63
    wdot           (t) float64 2.855e-07 0.0225 0.04499 ... -49.15 -46.12 -43.07
    c_m            (t) float64 -0.05797 -0.05873 -0.05849 ... -0.03054 -0.02537
    u_est          (t) float64 150.0 149.7 149.5 149.7 ... 149.6 149.5 149.5
    v_est          (t) float64 0.0 -5.895 -2.788 -3.39 ... 37.96 38.21 38.44
    w_est          (t) float64 0.0 0.3452 0.07328 ... -20.61 -21.1 -21.56
    C_alpha_u_est  (t) float64 0.6021 0.8656 0.755 ... 0.6066 0.6066 0.6065
    p_k1k1         (t, dim_0, dim_1) float64 0.001 0.0 ... -2.304e-07 1.453e-07
    iekf_i_count   (t) int64 0 100 19 19 10 9 12 14 8 7 ... 3 2 3 3 3 3 3 3 3 3
    vm_est         (t) float64 150.0 149.8 149.5 149.8 ... 155.7 155.8 155.9

In [48]:
plt.figure()
# f_data.u.plot()
f_data.C_alpha_u_est.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<Figure size 640x480 with 0 Axes>

In [49]:
plt.figure()
f_data.u_est.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<Figure size 640x480 with 0 Axes>

In [50]:
plt.figure()
f_data.V_m.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<Figure size 640x480 with 0 Axes>

In [51]:
plt.figure()
f_data.V_m.plot()
f_data.vm_est.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<Figure size 640x480 with 0 Axes>

In [52]:
plt.figure()
f_data.u_est.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<Figure size 640x480 with 0 Axes>